In [1]:
import pandas as pd
from google.colab import drive
import random
import numpy as np
import torch

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
user_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Khuad_최종플젝/silver_gold_half_with_tag.csv')
problem_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Khuad_최종플젝/problem_processed.csv')

In [3]:
user_df.shape

(2707133, 25)

In [4]:
user_df

,Unnamed: 0,id_to_index,handle,solvedCount,voteCount,class,rivalCount,reverseRivalCount,tier,rating,...,coins,stardusts,joinedAt,bannedUntil,proUntil,rank,isRival,isReverseRival,target,tag
0,0,1,cheon4050,240.0,0.0,5.0,0.0,0.0,15.0,1599.0,...,0.0,10155.0,2021-06-19T00:00:00.000Z,1970-01-01T00:00:00.000Z,1970-01-01T00:00:00.000Z,5997.0,0.0,0.0,0,"[['implementation', 'arithmetic', 'math']]"
1,1,1,cheon4050,240.0,0.0,5.0,0.0,0.0,15.0,1599.0,...,0.0,10155.0,2021-06-19T00:00:00.000Z,1970-01-01T00:00:00.000Z,1970-01-01T00:00:00.000Z,5997.0,0.0,0.0,3,[['dp']]
2,2,1,cheon4050,240.0,0.0,5.0,0.0,0.0,15.0,1599.0,...,0.0,10155.0,2021-06-19T00:00:00.000Z,1970-01-01T00:00:00.000Z,1970-01-01T00:00:00.000Z,5997.0,0.0,0.0,5,"[['dp', 'graphs', 'topological_sorting']]"
3,3,1,cheon4050,240.0,0.0,5.0,0.0,0.0,15.0,1599.0,...,0.0,10155.0,2021-06-19T00:00:00.000Z,1970-01-01T00:00:00.000Z,1970-01-01T00:00:00.000Z,5997.0,0.0,0.0,7,"[['bruteforcing', 'math']]"
4,4,1,cheon4050,240.0,0.0,5.0,0.0,0.0,15.0,1599.0,...,0.0,10155.0,2021-06-19T00:00:00.000Z,1970-01-01T00:00:00.000Z,1970-01-01T00:00:00.000Z,5997.0,0.0,0.0,8,"[['implementation', 'arithmetic', 'math']]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2707128,2707128,67299,yuiop7037,63.0,0.0,1.0,0.0,0.0,6.0,200.0,...,0.0,2130.0,2023-06-07T01:13:41.576Z,1970-01-01T00:00:00.000Z,1970-01-01T00:00:00.000Z,73263.0,0.0,0.0,0,"[['implementation', 'arithmetic', 'math']]"
2707129,2707129,67299,yuiop7037,63.0,0.0,1.0,0.0,0.0,6.0,200.0,...,0.0,2130.0,2023-06-07T01:13:41.576Z,1970-01-01T00:00:00.000Z,1970-01-01T00:00:00.000Z,73263.0,0.0,0.0,1,"[['implementation', 'arithmetic', 'math']]"
2707130,2707130,67299,yuiop7037,63.0,0.0,1.0,0.0,0.0,6.0,200.0,...,0.0,2130.0,2023-06-07T01:13:41.576Z,1970-01-01T00:00:00.000Z,1970-01-01T00:00:00.000Z,73263.0,0.0,0.0,8,"[['implementation', 'arithmetic', 'math']]"
2707131,2707131,67299,yuiop7037,63.0,0.0,1.0,0.0,0.0,6.0,200.0,...,0.0,2130.0,2023-06-07T01:13:41.576Z,1970-01-01T00:00:00.000Z,1970-01-01T00:00:00.000Z,73263.0,0.0,0.0,1487,[['implementation']]


## 데이터 전처리

In [5]:
df_user_problems = user_df[['id_to_index', 'target']]

In [6]:
df_user_problems

,id_to_index,target
0,1,0
1,1,3
2,1,5
3,1,7
4,1,8
...,...,...
2707128,67299,0
2707129,67299,1
2707130,67299,8
2707131,67299,1487


In [7]:
df_user_problems['solve'] = [1] * len(df_user_problems)
pivot_table = df_user_problems.pivot_table(index=["id_to_index"], columns=["target"], values="solve")
X = pivot_table.to_numpy()
X = np.nan_to_num(X)

<ipython-input-7-74f793fe73a8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_user_problems['solve'] = [1] * len(df_user_problems)


In [8]:
pivot_table

target,0,1,2,3,4,5,6,7,8,9,...,7604,7613,7614,7615,7620,7621,7622,7625,7626,7627
id_to_index,,,,,,,,,,,,,,,,,,,,,
1,1.0,NaN,NaN,1.0,NaN,1.0,NaN,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1.0,1.0,1.0,1.0,NaN,1.0,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67291,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67293,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67295,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
X.shape[1]

5539

## VAE 모델 생성

In [10]:
import tensorflow as tf
print(tf.__version__)

2.12.0


In [11]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2 as cv
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import shape,math
from tensorflow.keras import Input,layers,Model
from tensorflow.keras.losses import mse,binary_crossentropy
from tensorflow.keras.utils import plot_model

print(tf.__version__)

2.12.0


In [12]:
from tensorflow import shape,math
from tensorflow.keras import Input,layers,Model
from tensorflow.keras.losses import mse,binary_crossentropy

# network parameters
input_shape = X.shape[1]
original_dim= input_shape
intermediate_dim = 512
latent_dim = 2

In [13]:
def encoder():
  # 인코더의 입력층을 생성합니다.
  inputs = Input(shape=(input_shape,), name='input_shape')

  # 인코더의 hidden층을 생성합니다. 500개의 유닛을 사용했습니다.
  encoder_hidden = layers.Dense(intermediate_dim, activation='relu', name='encoder_hidden1')(inputs)

  # 평균(mean)과 표준편차(sigma)층을 정의합니다.
  # 이때 sigma 대신 log variance를 사용합니다. 이는 신경망의 출력은 음수를 가질 수 있지만 sigma는 항상 양수여야 하기 때문입니다.
  # 각각 2개의 유닛을 사용했습니다.
  z_mean = layers.Dense(latent_dim, name='z_mean')(encoder_hidden)
  z_log_var = layers.Dense(latent_dim, name='z_log_var')(encoder_hidden)

  # 평균과 표준편차를 래핑하여 Z_sampling층을 만듭니다.

  # Z 샘플링 함수 생성
  def sampling(args):
      z_mean, z_log_var = args
      batch = shape(z_mean)[0]
      dim = shape(z_mean)[1]

      # by default, random_normal has mean = 0 and std = 1.0
      # Reparameterization Trick사용을 위해 Gussian(=normal)분포에서 랜덤변수(sample) ε추출
      epsilon = tf.compat.v2.random.normal(shape=(batch, dim))
      return z_mean + tf.math.exp(0.5 * z_log_var) * epsilon

  #  layers.Lambda API 래핑에 사용할 함수와, 유닛수(n,)를 지정합니다.
  z_sampling = layers.Lambda(sampling, (latent_dim,), name='z_sample')([z_mean, z_log_var])

  # 하나의 입력과 다중충력을 포함하는 encoder 모델을 만듭니다.
  return Model(inputs,[z_mean,z_log_var,z_sampling], name='encoder')

encoder = encoder()

# 인코더 네트워크를 확인합니다.
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_shape (InputLayer)       [(None, 5539)]       0           []                               
                                                                                                  
 encoder_hidden1 (Dense)        (None, 512)          2836480     ['input_shape[0][0]']            
                                                                                                  
 z_mean (Dense)                 (None, 2)            1026        ['encoder_hidden1[0][0]']        
                                                                                                  
 z_log_var (Dense)              (None, 2)            1026        ['encoder_hidden1[0][0]']        
                                                                                            

In [14]:
def decoder():

  # 디코더의 입력층을 생성합니다. (Decoder의 입력은 latent입니다)
  input_z = Input(shape=(latent_dim,), name='input_z')

  # 디코더의 hidden층을 생성합니다. 인코더와 동일하게 500개의 유닛을 사용했습니다.
  decoder_hidden = layers.Dense(intermediate_dim, activation='relu', name='decoder_hidden')(input_z)

  # 디코더의 출력층은 인코더 입력벡터 수만큼 유닛을 사용합니다.
  outputs = layers.Dense(original_dim, activation='sigmoid',name='output')(decoder_hidden)

  return Model(input_z, outputs, name='decoder')

decoder = decoder()

# 디코더의 네트워크 확인.
decoder.summary()

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_z (InputLayer)        [(None, 2)]               0         
                                                                 
 decoder_hidden (Dense)      (None, 512)               1536      
                                                                 
 output (Dense)              (None, 5539)              2841507   
                                                                 
Total params: 2,843,043
Trainable params: 2,843,043
Non-trainable params: 0
_________________________________________________________________


In [15]:
def vae():
  # vae는 입력으로 이미지로 들어와 encoder를 통해 z_sampling 되어 decoder로 출력됩니다.
  inputs = Input(shape=(input_shape,), name='input_shape')
  outputs = decoder(encoder(inputs)[2]) #[0]:z_mean, [1]:z_log_var,[2]:z_sampling

  return Model(inputs,outputs, name='vae_mlp')

#VAE 모델 정의
model = vae()

#모델 네트워크 확인
model.summary()

Model: "vae_mlp"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_shape (InputLayer)    [(None, 5539)]            0         
                                                                 
 encoder (Functional)        [(None, 2),               2838532   
                              (None, 2),                         
                              (None, 2)]                         
                                                                 
 decoder (Functional)        (None, 5539)              2843043   
                                                                 
Total params: 5,681,575
Trainable params: 5,681,575
Non-trainable params: 0
_________________________________________________________________


In [16]:
# Hyperparameters
num_epochs = 100
batch_size = 20
learning_rate = 1e-3

In [17]:
# 모델 학습 loss, optimizer 정의
adam = tf.keras.optimizers.Adam(learning_rate=learning_rate)
def vae_loss(x,recon_x):
    # (1)Reconstruct loss (Marginal_likelihood) : Cross-entropy
    z_mean,z_log_var,z_sampling = encoder(x)
    recon_x=decoder(z_sampling)
    reconstruction_loss = binary_crossentropy(x,recon_x)
    #reconstruction_loss = mse(inputs, outputs)
    reconstruction_loss *= original_dim
    # (2) KL divergence(Latent_loss)
    kl_loss = 0.5 * tf.reduce_sum(tf.square(z_mean)+ tf.exp(z_log_var)- z_log_var -1, 1)
    return tf.reduce_mean(reconstruction_loss + kl_loss) #ELBO(=VAE_loss)

model.compile(optimizer=adam,loss=vae_loss)

In [18]:
 #모델 학습
hist = model.fit(X, X, epochs=num_epochs, batch_size=batch_size )

Epoch 1/100
1678/1678 [==============================] - 14s 5ms/step - loss: 216.6418
Epoch 2/100
1678/1678 [==============================] - 7s 4ms/step - loss: 157.1080
Epoch 3/100
1678/1678 [==============================] - 8s 5ms/step - loss: 154.8559
Epoch 4/100
1678/1678 [==============================] - 7s 4ms/step - loss: 153.8029
Epoch 5/100
1678/1678 [==============================] - 8s 5ms/step - loss: 152.9823
Epoch 6/100
1678/1678 [==============================] - 7s 4ms/step - loss: 152.3357
Epoch 7/100
1678/1678 [==============================] - 7s 4ms/step - loss: 151.7798
Epoch 8/100
1678/1678 [==============================] - 8s 5ms/step - loss: 151.3381
Epoch 9/100
1678/1678 [==============================] - 7s 4ms/step - loss: 150.9343
Epoch 10/100
1678/1678 [==============================] - 8s 5ms/step - loss: 150.5539
Epoch 11/100
1678/1678 [==============================] - 7s 4ms/step - loss: 150.2717
Epoch 12/100
1678/1678 [===========================

In [45]:
new_X = model.predict(X[0:10002])

313/313 [==============================] - 1s 2ms/step


In [46]:
result_df = pd.DataFrame(new_X)

In [47]:
result_df

,0,1,2,3,4,5,6,7,8,9,...,5529,5530,5531,5532,5533,5534,5535,5536,5537,5538
0,0.573954,0.398919,0.198835,0.641327,0.092443,0.255036,3.014077e-03,0.021589,0.338206,0.071276,...,0.000000e+00,2.819873e-11,1.301137e-03,2.566881e-23,1.964677e-06,6.784380e-06,2.849322e-22,2.642576e-03,6.948741e-04,3.732692e-03
1,0.991694,0.972153,0.500082,0.775241,0.221463,0.288039,7.203572e-03,0.028133,0.939302,0.301086,...,9.802657e-08,1.664511e-03,8.790959e-04,2.543792e-09,9.172946e-04,1.181687e-03,2.944465e-12,4.901141e-03,2.551795e-03,1.608927e-03
2,0.990970,0.977780,0.523293,0.880212,0.238238,0.249153,2.154555e-03,0.023725,0.938785,0.389789,...,2.586139e-20,3.315716e-03,3.717771e-03,4.486414e-21,1.918134e-03,6.579183e-03,9.537864e-12,6.142632e-03,3.073733e-03,3.916882e-03
3,0.999983,0.999738,0.746449,0.925664,0.463902,0.285949,4.886198e-03,0.037603,0.992148,0.757670,...,4.050690e-08,9.354511e-03,5.600304e-03,9.062147e-36,6.219144e-03,7.214321e-03,5.883847e-06,7.767103e-04,2.474076e-04,1.141633e-03
4,0.992881,0.992669,0.708738,0.958236,0.202128,0.226540,3.146881e-03,0.056613,0.986768,0.169430,...,9.702784e-27,2.993654e-04,9.812411e-04,3.539553e-14,3.210337e-05,8.198968e-14,2.843659e-19,4.109945e-16,1.629502e-06,6.806273e-19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9997,0.969798,0.914867,0.065497,0.043642,0.016826,0.000546,7.196931e-04,0.000167,0.862014,0.089266,...,1.441590e-20,4.658559e-18,1.437640e-23,1.463765e-22,2.033058e-04,3.827172e-05,3.760958e-28,4.328833e-06,0.000000e+00,1.763424e-29
9998,0.192521,0.093026,0.027689,0.057866,0.007127,0.008200,2.411316e-03,0.004035,0.023607,0.007229,...,0.000000e+00,8.073747e-33,2.554694e-20,4.892638e-35,1.302893e-15,2.111715e-17,3.556086e-32,3.064196e-15,8.757202e-37,3.199659e-22
9999,0.514739,0.347654,0.095356,0.144185,0.026087,0.001947,6.007356e-04,0.001219,0.225990,0.035912,...,1.371618e-36,1.506610e-09,8.541738e-18,7.730513e-27,1.461782e-04,5.470476e-06,1.005256e-28,1.319598e-06,5.193029e-35,1.365008e-17
10000,0.964527,0.940375,0.116841,0.036548,0.006130,0.001210,1.048586e-16,0.000140,0.953363,0.009727,...,0.000000e+00,5.144553e-32,2.367592e-23,0.000000e+00,3.508963e-06,1.755300e-33,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


In [ ]:
'''class MultiVAE(nn.Module):
    """
    Container module for Multi-VAE.

    Multi-VAE : Variational Autoencoder with Multinomial Likelihood
    See Variational Autoencoders for Collaborative Filtering
    https://arxiv.org/abs/1802.05814
    """

    def __init__(self, p_dims, q_dims=None, dropout=0.5):
        super(MultiVAE, self).__init__()
        self.p_dims = p_dims
        if q_dims:
            assert q_dims[0] == p_dims[-1], "In and Out dimensions must equal to each other"
            assert q_dims[-1] == p_dims[0], "Latent dimension for p- and q- network mismatches."
            self.q_dims = q_dims
        else:
            self.q_dims = p_dims[::-1]

        # Last dimension of q- network is for mean and variance
        temp_q_dims = self.q_dims[:-1] + [self.q_dims[-1] * 2]
        self.q_layers = nn.ModuleList([nn.Linear(d_in, d_out) for
            d_in, d_out in zip(temp_q_dims[:-1], temp_q_dims[1:])])
        self.p_layers = nn.ModuleList([nn.Linear(d_in, d_out) for
            d_in, d_out in zip(self.p_dims[:-1], self.p_dims[1:])])

        self.drop = nn.Dropout(dropout)
        self.init_weights()

    # Gaussian Distribution
    def forward(self, input):
        mu, logvar = self.encode(input)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

    # normal distribution 을 따르지 않은 input 에 대해 편향을 줄여줌
    # 예 ) 평점을 대체로 높게 주는 사용자와 낮게 주는 사용자의 편향을 줄임
    def encode(self, input):
        h = F.normalize(input)
        h = self.drop(h)
        for i, layer in enumerate(self.q_layers):
            h = layer(h)
            if i != len(self.q_layers) - 1:
                h = F.tanh(h)
            else:
                mu = h[:, :self.q_dims[-1]]
                logvar = h[:, self.q_dims[-1]:]
        return mu, logvar

    # Gaussian 분포의 표준 편차를 구하는 식 ( 분산의 제곱근 )
    def reparameterize(self, mu, logvar):
        if self.training:
            std = torch.exp(0.5 * logvar)
            eps = torch.randn_like(std)
            return eps.mul(std).add_(mu)
        else:
            return mu

    def decode(self, z):
        h = z
        for i, layer in enumerate(self.p_layers):
            h = layer(h)
            if i != len(self.p_layers) - 1:
                h = F.tanh(h)
        return h

    def init_weights(self):
        for layer in self.q_layers:
            # Xavier Initialization for weights
            size = layer.weight.size()
            fan_out = size[0]
            fan_in = size[1]
            std = np.sqrt(2.0/(fan_in + fan_out))
            layer.weight.data.normal_(0.0, std)

            # Normal Initialization for Biases
            layer.bias.data.normal_(0.0, 0.001)

        for layer in self.p_layers:
            # Xavier Initialization for weights
            size = layer.weight.size()
            fan_out = size[0]
            fan_in = size[1]
            std = np.sqrt(2.0/(fan_in + fan_out))
            layer.weight.data.normal_(0.0, std)

            # Normal Initialization for Biases
            layer.bias.data.normal_(0.0, 0.001)

def loss_function(recon_x, x, mu, logvar, anneal=1.0):
    # BCE = F.binary_cross_entropy(recon_x, x)
    BCE = -torch.mean(torch.sum(F.log_softmax(recon_x, 1) * x, -1))
    KLD = -0.5 * torch.mean(torch.sum(1 + logvar - mu.pow(2) - logvar.exp(), dim=1))
    return BCE + anneal * KLD
    '''